In [6]:
import wandb
import json
import yaml
import os, sys

In [7]:
wb_config_name = 'wb_config_1.yaml'
wb_project = 'videomae_finetuning'

sweep_id_dict_name = 'sweep_id_dict.json'

In [8]:
with open(sweep_id_dict_name) as f:
    sweep_dict = json.load(f)
    if wb_config_name in sweep_dict:
        sweep_id = sweep_dict[wb_config_name]
    else:
        # Initialize sweep by passing in config
        config = yaml.load(open(wb_config_name), Loader=yaml.Loader)
        sweep_id = wandb.sweep(sweep=config,
                               project=wb_project,
                               entity='flatiron-scipt')
        sweep_dict[wb_config_name] = sweep_id
        with open(sweep_id_dict_name, 'w') as f:
            json.dump(sweep_dict, f, indent=4)
        print(f"New sweep_id created for config {wb_config_name}: {sweep_id}. Added to the sweep_ids dictionary!")
        sys.exit(0)

In [9]:
with open('finetune_template.sh', 'r') as f:
    finetune_script_template = f.read()

In [5]:
run_id_start = 8
nb_runs = 42
for run_id in range(run_id_start, run_id_start + nb_runs):
    finetune_script = finetune_script_template.replace('{{run_id}}', str(run_id))
    finetune_script = finetune_script.replace('{{sweep_id}}', sweep_id)
    with open(f'jobs/finetune_{run_id}.sh', 'w') as f:
        f.write(finetune_script)
    os.system(f'sbatch jobs/finetune_{run_id}.sh')

Submitted batch job 2620650
Submitted batch job 2620651
Submitted batch job 2620652
Submitted batch job 2620653
Submitted batch job 2620654
Submitted batch job 2620655
Submitted batch job 2620656
Submitted batch job 2620657
Submitted batch job 2620658
Submitted batch job 2620659
Submitted batch job 2620660
Submitted batch job 2620661
Submitted batch job 2620662
Submitted batch job 2620663
Submitted batch job 2620664
Submitted batch job 2620665
Submitted batch job 2620666
Submitted batch job 2620667
Submitted batch job 2620668
Submitted batch job 2620669
Submitted batch job 2620670
Submitted batch job 2620671
Submitted batch job 2620672
Submitted batch job 2620673
Submitted batch job 2620674
Submitted batch job 2620675
Submitted batch job 2620676
Submitted batch job 2620677
Submitted batch job 2620678
Submitted batch job 2620679
Submitted batch job 2620680
Submitted batch job 2620681
Submitted batch job 2620682
Submitted batch job 2620683
Submitted batch job 2620684
Submitted batch job 